In [ ]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from src import detect_faces, show_bboxes
from PIL import Image

import torch
from torchvision import transforms, datasets
import numpy as np
import os

In [3]:
dataset_path = '../Dataset/emotiw/'
face_coordinates_directory = '../Dataset/FaceCoordinates/'
processed_dataset_path = '../Dataset/CroppedFaces/'

In [4]:
# data_transforms = {
#     'train' : transforms.Compose([transforms.ToPILImage()]),
    
#     'val' : transforms.Compose([transforms.ToPILImage()])
# }


# image_datasets = {x : datasets.ImageFolder(os.path.join(dataset_path, x), data_transforms[x])
#                     for x in ['train', 'val']}

image_datasets = {x : datasets.ImageFolder(os.path.join(dataset_path, x))
                    for x in ['train', 'val']}

# dataloaders = { x : torch.utils.data.DataLoader(image_datasets[x], batch_size = 1, shuffle=True, num_workers= 4)
#                    for x in ['train', 'val']}

# dataset_sizes = {x : len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
class_names

['Negative', 'Neutral', 'Positive']

In [6]:
# type(train_data[0:5])

# # neg_images = []
# # neutral_images = []
# # pos_images = []
# # for i in range(len(image_datasets['train'])):
# #     label = image_datasets['train'][i][1]
# #     if label == 0:
# #         neg_images.append(image_datasets['train'][i][0])
# #         print('neg')
# #     elif label == 1:
# #         neutral_images.append(image_datasets['train'][i][0])
# #         print('neut')
# #     else:
# #         pos_images.append(image_datasets['train'][i][0])
# #         print('pos')

In [7]:
# neg_images = 2758
# neut_images = 3080
# pos_images = 3997

# train_neg = image_datasets['train'][0:neg_images][0]
# train_neut = image_datasets['train'][neg_images: neg_images + neut_images][0]
# train_pos = image_datasets['train'][neg_images + neut_images : -1][0]

# val_neg = image_datasets['val'][0:neg_images][0]
# val_neut = image_datasets['val'][neg_images: neg_images + neut_images][0]
# val_pos = image_datasets['val'][neg_images + neut_images : -1][0]

In [8]:
training_dataset = image_datasets['train']
validation_dataset = image_datasets['val']

In [9]:
neg_train = sorted(os.listdir(dataset_path + 'train/Negative/'))
neu_train = sorted(os.listdir(dataset_path + 'train/Neutral/'))
pos_train = sorted(os.listdir(dataset_path + 'train/Positive/'))

neg_val = sorted(os.listdir(dataset_path + 'val/Negative/'))
neu_val = sorted(os.listdir(dataset_path + 'val/Neutral/'))
pos_val = sorted(os.listdir(dataset_path + 'val/Positive/'))

In [10]:
neg_train_filelist = [x.split('.')[0] for x in neg_train]
neu_train_filelist = [x.split('.')[0] for x in neu_train]
pos_train_filelist = [x.split('.')[0] for x in pos_train]

neg_val_filelist = [x.split('.')[0] for x in neg_val]
neu_val_filelist = [x.split('.')[0] for x in neu_val]
pos_val_filelist = [x.split('.')[0] for x in pos_val]

In [11]:
print(neg_train_filelist[:10])
print(neu_train_filelist[:10])
print(pos_train_filelist[:10])

print(neg_val_filelist[:10])
print(neu_val_filelist[:10])
print(pos_val_filelist[:10])

['neg_1', 'neg_10', 'neg_100', 'neg_1000', 'neg_1001', 'neg_1002', 'neg_1003', 'neg_1004', 'neg_1005', 'neg_1006']
['neu_1', 'neu_10', 'neu_100', 'neu_1000', 'neu_1001', 'neu_1002', 'neu_1003', 'neu_1004', 'neu_1005', 'neu_1006']
['pos_1', 'pos_10', 'pos_100', 'pos_1000', 'pos_1001', 'pos_1002', 'pos_1003', 'pos_1004', 'pos_1005', 'pos_1006']
['neg_1', 'neg_10', 'neg_100', 'neg_1000', 'neg_1001', 'neg_1002', 'neg_1003', 'neg_1004', 'neg_1005', 'neg_1006']
['neu_1', 'neu_10', 'neu_100', 'neu_1000', 'neu_1001', 'neu_1002', 'neu_1003', 'neu_1004', 'neu_1005', 'neu_1006']
['pos_1', 'pos_10', 'pos_100', 'pos_1000', 'pos_1001', 'pos_1002', 'pos_1003', 'pos_1004', 'pos_1005', 'pos_1006']


In [12]:
train_filelist = neg_train_filelist + neu_train_filelist + pos_train_filelist
val_filelist = neg_val_filelist + neu_val_filelist + pos_val_filelist

In [13]:
print(len(training_dataset))
print(len(validation_dataset))

9815
4346


In [14]:
# pos_train_count = 0
# neg_train_count = 0
# neut_train_count = 0

In [15]:
# for i in range(len(image_datasets['train'])):
# # for i in range(1):
#     image, label = image_datasets['train'][i]
# #     bounding_boxes, landmarks = detect_faces(image)
#     face_list = []
#     if label == 0:
#         bbox_lm = np.load('../Dataset/FaceFeatures/train/Negative/neg_' + str(neg_train_count + 1) +'.npz')
#         bounding_boxes = bbox_lm['a']
#         landmarks = bbox_lm['b']
        
#         for j in range(len(bounding_boxes)):
#             bbox_coordinates = bounding_boxes[j]
#             img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
# #             if(len(face_list) != 0 and face_list[-1].shape[0] == int(bbox_coordinates[2] - bbox_coordinates[0])):
# #                 img_face = image.crop((bbox_coordinates[0] -1, bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
#             img_face = np.array(img_face)
            
#             if len(face_list) != 0:
#                 if img_face.shape[0] == face_list[-1].shape[0]:
#                     img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
#                     img_face = np.array(img_face)
            
#             face_list.append(img_face)
#         face_list = np.asarray(face_list)
#         np.savez('../Dataset/SingleFaceFeatures/train/Negative/neg_' + str(neg_train_count + 1), a = face_list)
        
#         neg_train_count += 1
# #         print('neg' + str(neg_train_count+1))
        
#     elif label == 1:
#         bbox_lm = np.load('../Dataset/FaceFeatures/train/Neutral/neu_' + str(neut_train_count + 1) +'.npz')
#         bounding_boxes = bbox_lm['a']
#         landmarks = bbox_lm['b']
        
#         for j in range(len(bounding_boxes)):
#             bbox_coordinates = bounding_boxes[j]
#             img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
# #             if(len(face_list) != 0 and face_list[-1].shape[0] == int(bbox_coordinates[2] - bbox_coordinates[0])):
# #                 img_face = image.crop((bbox_coordinates[0] -1, bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
#             img_face = np.array(img_face)
            
#             if len(face_list) != 0:
#                 if img_face.shape[0] == face_list[-1].shape[0]:
#                     img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
#                     img_face = np.array(img_face)
            
#             face_list.append(img_face)
#         face_list = np.asarray(face_list)
#         np.savez('../Dataset/SingleFaceFeatures/train/Neutral/neu_' + str(neut_train_count + 1), a = face_list)
        
#         neut_train_count += 1

        
#     else:
#         bbox_lm = np.load('../Dataset/FaceFeatures/train/Positive/pos_' + str(pos_train_count + 1) +'.npz')
#         bounding_boxes = bbox_lm['a']
#         landmarks = bbox_lm['b']
        
#         for j in range(len(bounding_boxes)):
#             bbox_coordinates = bounding_boxes[j]
#             img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
# #             if (len(face_list) != 0 and face_list[-1].shape[0] == bbox_coordinates[2] - bbox_coordinates[0]):
# #                 img_face = image.crop((bbox_coordinates[0] -1, bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
#             img_face = np.array(img_face)

#             if len(face_list) != 0:
#                 if img_face.shape[0] == face_list[-1].shape[0]:
#                     img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
#                     img_face = np.array(img_face)
            
#             face_list.append(img_face)
#         face_list = np.asarray(face_list)
#         np.savez('../Dataset/SingleFaceFeatures/train/Positive/pos_' + str(pos_train_count + 1), a = face_list)
        
#         pos_train_count += 1
#     if i % 100 == 0:
#         print(i)

In [ ]:
for i in range(len(training_dataset)):
# for i in range(10):
    image, label = training_dataset[i]
#     bounding_boxes, landmarks = detect_faces(image)
    face_list = []
    landmarks_new_coordinates = []
    if label == 0:
        bbox_lm = np.load(face_coordinates_directory + 'train/Negative/' + train_filelist[i] +'.npz')
        bounding_boxes = bbox_lm['a']
        if(bounding_boxes.size == 0):
            print("No bounding boxes for " + train_filelist[i] + ". Adding empty file for the same")
            np.savez(processed_dataset_path + 'train/Negative' + train_filelist[i], a = np.zeros(1), b = np.zeros(1))
            continue
        landmarks = bbox_lm['b']
        
        for j in range(len(bounding_boxes)):
            bbox_coordinates = bounding_boxes[j]
            landmark = landmarks[j]
            img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
#             if(len(face_list) != 0 and face_list[-1].shape[0] == int(bbox_coordinates[2] - bbox_coordinates[0])):
#                 img_face = image.crop((bbox_coordinates[0] -1, bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
            x = bbox_coordinates[0]
            y = bbox_coordinates[1]
            for k in range(5):
                landmark[k] -= x
                landmark[k+5] -= y
            img_face = np.array(img_face)
            landmark = np.array(landmark)
            
            
            if len(face_list) != 0:
                if img_face.shape[0] == face_list[-1].shape[0]:
                    img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
                    img_face = np.array(img_face)
                    landmark +=1
            
            face_list.append(img_face)
            landmarks_new_coordinates.append(landmark)
        face_list = np.asarray(face_list)
        landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
        np.savez(processed_dataset_path + 'train/Negative/' + train_filelist[i], a = face_list, b = landmarks_new_coordinates)
        
    elif label == 1:
        bbox_lm = np.load(face_coordinates_directory + 'train/Neutral/' + train_filelist[i] +'.npz')
        bounding_boxes = bbox_lm['a']
        if(bounding_boxes.size == 0):
            print("No bounding boxes for " + train_filelist[i] + ". Adding empty file for the same")
            np.savez(processed_dataset_path + 'train/Neutral' + train_filelist[i], a = np.zeros(1), b = np.zeros(1))
            continue
        landmarks = bbox_lm['b']
        
        for j in range(len(bounding_boxes)):
            bbox_coordinates = bounding_boxes[j]
            landmark = landmarks[j]
            img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
#             if(len(face_list) != 0 and face_list[-1].shape[0] == int(bbox_coordinates[2] - bbox_coordinates[0])):
#                 img_face = image.crop((bbox_coordinates[0] -1, bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
            x = bbox_coordinates[0]
            y = bbox_coordinates[1]
            for k in range(5):
                landmark[k] -= x
                landmark[k+5] -= y
            img_face = np.array(img_face)
            landmark = np.array(landmark)
            
            if len(face_list) != 0:
                if img_face.shape[0] == face_list[-1].shape[0]:
                    img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
                    img_face = np.array(img_face)
                    landmark += 1
            
            face_list.append(img_face)
            landmarks_new_coordinates.append(landmark)
        face_list = np.asarray(face_list)
        landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
        np.savez(processed_dataset_path + 'train/Neutral/' + train_filelist[i], a = face_list, b = landmarks_new_coordinates)
        
    else:
        bbox_lm = np.load(face_coordinates_directory + 'train/Positive/' + train_filelist[i] +'.npz')
        bounding_boxes = bbox_lm['a']
        if(bounding_boxes.size == 0):
            print("No bounding boxes for " + train_filelist[i] + ". Adding empty file for the same")
            np.savez(processed_dataset_path + 'train/Positive' + train_filelist[i], a = np.zeros(1), b = np.zeros(1))
            continue
        landmarks = bbox_lm['b']
        
        for j in range(len(bounding_boxes)):
            bbox_coordinates = bounding_boxes[j]
            landmark = landmarks[j]
            img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
#             if(len(face_list) != 0 and face_list[-1].shape[0] == int(bbox_coordinates[2] - bbox_coordinates[0])):
#                 img_face = image.crop((bbox_coordinates[0] -1, bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
            x = bbox_coordinates[0]
            y = bbox_coordinates[1]
            for k in range(5):
                landmark[k] -= x
                landmark[k+5] -= y
            img_face = np.array(img_face)
            landmark = np.array(landmark)
            
            if len(face_list) != 0:
                if img_face.shape[0] == face_list[-1].shape[0]:
                    img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
                    img_face = np.array(img_face)
                    landmark += 1
            
            face_list.append(img_face)
            landmarks_new_coordinates.append(landmark)
        face_list = np.asarray(face_list)
        landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
        np.savez(processed_dataset_path + 'train/Positive/' + train_filelist[i], a = face_list, b = landmarks_new_coordinates)
        
    if i % 100 == 0:
        print(i)

0
No bounding boxes for neg_10. Adding empty file for the same
100
200
No bounding boxes for neg_118. Adding empty file for the same
No bounding boxes for neg_123. Adding empty file for the same
300
400
No bounding boxes for neg_14. Adding empty file for the same
No bounding boxes for neg_141. Adding empty file for the same
No bounding boxes for neg_144. Adding empty file for the same
500
600
700
800
900
1000
No bounding boxes for neg_190. Adding empty file for the same
1100
1200
1300
1400
1500
No bounding boxes for neg_2373. Adding empty file for the same
No bounding boxes for neg_2395. Adding empty file for the same
No bounding boxes for neg_2434. Adding empty file for the same
1600
1700
No bounding boxes for neg_254. Adding empty file for the same
No bounding boxes for neg_261. Adding empty file for the same
1800
No bounding boxes for neg_2671. Adding empty file for the same
No bounding boxes for neg_2684. Adding empty file for the same
No bounding boxes for neg_2694. Adding empty f

In [ ]:
# for i in range(len(validation_dataset)):
# # for i in range(10):
#     image, label = validation_dataset[i]
# #     bounding_boxes, landmarks = detect_faces(image)
#     face_list = []
#     landmarks_new_coordinates = []
#     if label == 0:
#         bbox_lm = np.load(face_coordinates_directory + 'val/Negative/' + val_filelist[i] +'.npz')
#         bounding_boxes = bbox_lm['a']
#         if(bounding_boxes.size == 0):
#             print("No bounding boxes for " + val_filelist[i] + ". Adding empty file for the same")
#             np.savez(processed_dataset_path + 'val/Negative' + val_filelist[i], a = np.zeros(1), b = np.zeros(1))
#             continue
#         landmarks = bbox_lm['b']
        
#         for j in range(len(bounding_boxes)):
#             bbox_coordinates = bounding_boxes[j]
#             landmark = landmarks[j]
#             img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
# #             if(len(face_list) != 0 and face_list[-1].shape[0] == int(bbox_coordinates[2] - bbox_coordinates[0])):
# #                 img_face = image.crop((bbox_coordinates[0] -1, bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
#             x = bbox_coordinates[0]
#             y = bbox_coordinates[1]
#             for k in range(5):
#                 landmark[k] -= x
#                 landmark[k+5] -= y
#             img_face = np.array(img_face)
#             landmark = np.array(landmark)
            
            
#             if len(face_list) != 0:
#                 if img_face.shape[0] == face_list[-1].shape[0]:
#                     img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
#                     img_face = np.array(img_face)
#                     landmark +=1
            
#             face_list.append(img_face)
#             landmarks_new_coordinates.append(landmark)
#         face_list = np.asarray(face_list)
#         landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
#         np.savez(processed_dataset_path + 'val/Negative/' + val_filelist[i], a = face_list, b = landmarks_new_coordinates)
        
#     elif label == 1:
#         bbox_lm = np.load(face_coordinates_directory + 'val/Neutral/' + val_filelist[i] +'.npz')
#         bounding_boxes = bbox_lm['a']
#         if(bounding_boxes.size == 0):
#             print("No bounding boxes for " + val_filelist[i] + ". Adding empty file for the same")
#             np.savez(processed_dataset_path + 'val/Neutral' + val_filelist[i], a = np.zeros(1), b = np.zeros(1))
#             continue
#         landmarks = bbox_lm['b']
        
#         for j in range(len(bounding_boxes)):
#             bbox_coordinates = bounding_boxes[j]
#             landmark = landmarks[j]
#             img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
# #             if(len(face_list) != 0 and face_list[-1].shape[0] == int(bbox_coordinates[2] - bbox_coordinates[0])):
# #                 img_face = image.crop((bbox_coordinates[0] -1, bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
#             x = bbox_coordinates[0]
#             y = bbox_coordinates[1]
#             for k in range(5):
#                 landmark[k] -= x
#                 landmark[k+5] -= y
#             img_face = np.array(img_face)
#             landmark = np.array(landmark)
            
#             if len(face_list) != 0:
#                 if img_face.shape[0] == face_list[-1].shape[0]:
#                     img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
#                     img_face = np.array(img_face)
#                     landmark += 1
            
#             face_list.append(img_face)
#             landmarks_new_coordinates.append(landmark)
#         face_list = np.asarray(face_list)
#         landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
#         np.savez(processed_dataset_path + 'val/Neutral/' + val_filelist[i], a = face_list, b = landmarks_new_coordinates)
        
#     else:
#         bbox_lm = np.load(face_coordinates_directory + 'val/Positive/' + val_filelist[i] +'.npz')
#         bounding_boxes = bbox_lm['a']
#         if(bounding_boxes.size == 0):
#             print("No bounding boxes for " + val_filelist[i] + ". Adding empty file for the same")
#             np.savez(processed_dataset_path + 'val/Positive' + val_filelist[i], a = np.zeros(1), b = np.zeros(1))
#             continue
#         landmarks = bbox_lm['b']
        
#         for j in range(len(bounding_boxes)):
#             bbox_coordinates = bounding_boxes[j]
#             landmark = landmarks[j]
#             img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
# #             if(len(face_list) != 0 and face_list[-1].shape[0] == int(bbox_coordinates[2] - bbox_coordinates[0])):
# #                 img_face = image.crop((bbox_coordinates[0] -1, bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
#             x = bbox_coordinates[0]
#             y = bbox_coordinates[1]
#             for k in range(5):
#                 landmark[k] -= x
#                 landmark[k+5] -= y
#             img_face = np.array(img_face)
#             landmark = np.array(landmark)
            
#             if len(face_list) != 0:
#                 if img_face.shape[0] == face_list[-1].shape[0]:
#                     img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
#                     img_face = np.array(img_face)
#                     landmark += 1
            
#             face_list.append(img_face)
#             landmarks_new_coordinates.append(landmark)
#         face_list = np.asarray(face_list)
#         landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
#         np.savez(processed_dataset_path + 'val/Positive/' + val_filelist[i], a = face_list, b = landmarks_new_coordinates)
        
#     if i % 100 == 0:
#         print(i)

In [ ]:
# len(bounding_boxes)

In [ ]:
# landmarks[0]  # x1, x2, x3, x4, x5, y1, y2, y3, y4, y5

# Test unit

In [ ]:
image = image_datasets['train'][155][0]
bounding_boxes, landmarks = detect_faces(image)

In [ ]:
show_bboxes(image, bounding_boxes, landmarks)

In [ ]:
print(len(bounding_boxes))
print(len(landmarks))

In [ ]:
face_list = []

for j in range(len(bounding_boxes)):
    bbox_coordinates = bounding_boxes[j]
    img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
    img_face = np.array(img_face)
    print(img_face.shape)
    face_list.append(img_face)
    print(face_list)

face_list = np.asaarray(face_list, dtype=object)
np.savez('test', a = face_list)
        

In [ ]:
face_list.shape

In [ ]:
# img = image_datasets['train'][155][0]
img1 = image.crop((bbc[0], bbc[1], bbc[2], bbc[3]))

In [ ]:
img1

In [ ]:
img_np = np.array(img1)

In [ ]:
np.savez('./img', a = img_np)

In [ ]:
images = np.load('../Dataset/SingleFaceFeatures/train/Negative/neg_1.npz')['a']

In [ ]:
image = images[3]

In [ ]:
image = Image.fromarray(image)

In [ ]:
image

In [ ]:
a = [np.array([1, 2, 3]),np.array([1, 2])]
a = np.array(a)

In [ ]:
a

In [ ]:
arr = np.load('./neg_151.npz')['a']

In [ ]:
print(arr[0].shape)
print(arr[1].shape)

In [ ]:
arr = np.zeros(0)

In [ ]:
arr.size

In [ ]:
#Check if extracted faces and shifted landmarks are correct

In [ ]:
bbox_lm = np.load('../Dataset/CroppedFaces/train/Negative/neg_' + str(1004) +'.npz')

In [ ]:
bb = bbox_lm['a']
lm_test = bbox_lm['b']
lm_test[1].shape

In [ ]:
img_testing = Image.fromarray(bb[2])

show_bboxes(img_testing, np.array([[0, 0, bb[2].shape[0], bb[2].shape[0]]]), np.array([lm_test[2]]))